# Objetivo_1.2

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import altair as alt
import datos
from dash import dash_table
from dash.dash_table.Format import Group
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import seaborn as sns
import warnings

# Ignorar advertencias de tipo FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)


## Visualizaciones por Agrupaciones 
+ 'colesterol_total(mg/dl)'
+ 'cLDL(mg/dl)'
+ 'HDL(mg/dl)'
+ 'trigliceridos(mg/dl)'

### Histogramas y Boxplots

In [29]:
columnas = ['colesterol_total(mg/dl)', 'cLDL(mg/dl)', 'HDL(mg/dl)', 'trigliceridos(mg/dl)']
agrupaciones = ['sexo', '¿tiene_antecedentes_con_padre?', '¿tiene_antecedentes_con_madre?', '¿tiene_antecedentes_con_herman?', 'diabetes mellitus', 'hipercolesterolemia', 'dislipidemia']

for columna in columnas:
    for agrupacion in agrupaciones:
        # Gráfico de caja (box plot)
        fig_0 = px.box(datos.datos, x=agrupacion, y=columna, title=f'{columna} por {agrupacion}')
        fig_0.show()

        # Histograma
        fig_1 = px.histogram(datos.datos, x=columna, color=agrupacion, nbins=10, opacity=0.6, histnorm='density', title=f'Histograma de {columna} por {agrupacion}')
        fig_1.show()

### Tablas de frecuencia

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd


In [4]:
intervalos = {
    'colesterol_total(mg/dl)': [0, 158, 200, 239, 278, max(datos.datos['colesterol_total(mg/dl)'])],
    'cLDL(mg/dl)': [0, 35, 85, max(datos.datos['cLDL(mg/dl)'])],
    'HDL(mg/dl)': [0, 35, 46, 50, 62, max(datos.datos['HDL(mg/dl)'])],
    'trigliceridos(mg/dl)': [0, 100, 150, 200, 250, max(datos.datos['trigliceridos(mg/dl)'])],
}


def plot_frecuencias_general(df, column):
    intervalo = intervalos[column]
    edades_clasificadas = pd.cut(df[column], bins=intervalo)
    frecuencia = edades_clasificadas.value_counts().sort_index().reset_index()
    frecuencia.columns = ['Intervalo', 'Frecuencia']
    frecuencia['Intervalo'] = frecuencia['Intervalo'].astype(str)  # Convertir Intervalo a cadena
    
    fig = px.bar(frecuencia, x='Intervalo', y='Frecuencia', 
                 title=f'Frecuencias de {column}',
                 labels={'Intervalo': 'Intervalo', 'Frecuencia': 'Frecuencia'})
    fig.update_xaxes(tickangle=45)
    fig.show()

def plot_tabla_agrupada(df, column, agrupacion):
    intervalo = intervalos[column]
    df[f'{column}_rango'] = pd.cut(df[column], bins=intervalo)
    tabla_agrupada = df.groupby([agrupacion, f'{column}_rango']).size().reset_index(name='Frecuencia')
    tabla_agrupada[f'{column}_rango'] = tabla_agrupada[f'{column}_rango'].astype(str)  # Convertir Intervalo a cadena
    tabla_pivotada = tabla_agrupada.pivot_table(
        index=f'{column}_rango',
        columns=agrupacion,
        values='Frecuencia',
        fill_value=0
    ).reset_index()
    
    fig = px.imshow(tabla_pivotada.set_index(f'{column}_rango').T, 
                    labels={'color': 'Frecuencia'},
                    title=f'Tabla de frecuencias de {column} por {agrupacion}')
    fig.update_layout(yaxis_title='Categoría', xaxis_title='Intervalo')
    fig.show()

# Graficar para cada variable
for columna in intervalos.keys():
    plot_frecuencias_general(datos.datos, columna)
    
    for agrupacion in agrupaciones:
        plot_tabla_agrupada(datos.datos, columna, agrupacion)


## Por edad agrupada

In [5]:
edad_intervals = [0, 49, 59, 79, 150]
edad_labels = ['<50', '50-59', '60-79', '>80']
datos.datos['edad_agrupada'] = pd.cut(datos.datos['edad_calculada'], bins=edad_intervals, labels=edad_labels, right=True)

# Calcular frecuencias
frecuencias = datos.datos['edad_agrupada'].value_counts().sort_index()
tabla_frecuencias = pd.DataFrame({
    'Intervalo': frecuencias.index.astype(str),
    'Frecuencia': frecuencias.values
})
import plotly.graph_objects as go

def plot_frecuencias_general(df, column):
    intervalo = intervalos[column]
    edades_clasificadas = pd.cut(df[column], bins=intervalo)
    frecuencia = edades_clasificadas.value_counts().sort_index()
    
    df_frecuencia = pd.DataFrame({
        'Intervalo': frecuencia.index.astype(str),
        'Frecuencia': frecuencia.values
    })
    
    fig = px.bar(df_frecuencia, x='Intervalo', y='Frecuencia', 
                 title=f'Frecuencias de {column}',
                 labels={'Intervalo': 'Intervalo', 'Frecuencia': 'Frecuencia'})
    fig.update_xaxes(tickangle=45)
    fig.show()

# Definir la función para graficar tablas agrupadas
def plot_tabla_agrupada(df, column, agrupacion):
    intervalo = intervalos[column]
    df[f'{column}_rango'] = pd.cut(df[column], bins=intervalo)
    tabla_agrupada = df.groupby([agrupacion, f'{column}_rango']).size().reset_index(name='Frecuencia')
    tabla_agrupada[f'{column}_rango'] = tabla_agrupada[f'{column}_rango'].astype(str)
    tabla_pivotada = tabla_agrupada.pivot_table(
        index=f'{column}_rango',
        columns=agrupacion,
        values='Frecuencia',
        fill_value=0
    ).reset_index()

    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_pivotada.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_pivotada[col] for col in tabla_pivotada.columns], fill_color='lavender', align='left')
    )])
    
    fig.update_layout(title_text=f'Tabla de frecuencias de {column} por {agrupacion}')
    fig.show()

# Graficar para cada variable
for columna in intervalos.keys():
    plot_frecuencias_general(datos.datos, columna)
    
    for agrupacion in agrupaciones:
        plot_tabla_agrupada(datos.datos, columna, agrupacion)
